In [56]:
import pandas as pd
import json
from transformers import pipeline

In [57]:
df = pd.read_excel('04_Consolidado.xlsx')
df['eval'] = None

In [58]:
# Load a sentiment analysis pipeline
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# 1 star: Very negative sentiment.
# 2 stars: Negative sentiment.
# 3 stars: Neutral sentiment.
# 4 stars: Positive sentiment.
# 5 stars: Very positive sentiment.

In [59]:
for index, row in df.iterrows():
    if row['eval'] is not None:
        print(f'Skipped: {index}')
        continue

    oraciones = pd.DataFrame({'oracion' : json.loads(row['oraciones'])})
    oraciones['etiqueta'] = None
    oraciones['puntaje'] = None
    oraciones['estrellas'] = None
    
    for ind, oracion in oraciones.iterrows():
        result = classifier(oracion['oracion'])
        oraciones.at[ind,'etiqueta'] = result[0]['label']
        oraciones.at[ind,'estrellas'] = int(result[0]['label'][0])
        oraciones.at[ind,'puntaje'] = float(result[0]['score'])

    df.loc[index,'eval'] = oraciones.copy
    print(f'Wrote: {index}', end='\r')

In [ ]:
df.to_excel('05_Sentimiento.xlsx')